<a href="https://colab.research.google.com/github/ezgiiii/Data-Analysis-Projects/blob/master/Scouting_with_Scoutium_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is done using two datasets from Scoutium. I tried to compare the hold-out and cross validation method with score metrics.


Enjoy! 😀

**scoutium_attributes.csv**

task_response_id: The set of evaluations a scout made for a particular team

match_id: The id of a match

evaluator_id: The scout's id

player_id: The player's id

position_id: The player's position in a given match

        1:Goalkeeper
        2:Stopper
        3:Right Fielder
        4:Left Fielder
        5:Defensive Midfielder
        6:Centre Midfielder
        7:Right Wing
        8:Left Wing
        9:Attacking Midfielder
        10:Forward

analysis_id: The set of evaluations a scout made for a particular player

attribute_id: The id of a player's attribute

attribute_value: The point of a player's attribute

**scoutium_potential_labels.csv**

task_response_id: The set of evaluations a scout made for a particular team

match_id: The id of a match

evaluator_id: The scout's id

player_id: The player's id

potential_label: The final decision that a scout made for a  player.


In [49]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [4]:
df_=pd.read_csv("/gdrive/My Drive/Colab Notebooks/scoutium_attributes.csv",sep=";")
df_s=pd.read_csv("/gdrive/My Drive/Colab Notebooks/scoutium_potential_labels.csv",sep=";")

In [5]:
df_att=df_.copy()
df_pot=df_s.copy()

In [6]:
df_att.info

<bound method DataFrame.info of        task_response_id  match_id  evaluator_id  player_id  position_id  \
0                  4915     62935        177676    1361061            2   
1                  4915     62935        177676    1361061            2   
2                  4915     62935        177676    1361061            2   
3                  4915     62935        177676    1361061            2   
4                  4915     62935        177676    1361061            2   
...                 ...       ...           ...        ...          ...   
10725              5642     63032        151191    1909728            7   
10726              5642     63032        151191    1909728            7   
10727              5642     63032        151191    1909728            7   
10728              5642     63032        151191    1909728            7   
10729              5642     63032        151191    1909728            7   

       analysis_id  attribute_id  attribute_value  
0         12818

In [7]:
df_pot.head()

,task_response_id,match_id,evaluator_id,player_id,potential_label
0,4915,62935,177676,1361061,average
1,4915,62935,177676,1361626,highlighted
2,4915,62935,177676,1361858,average
3,4915,62935,177676,1362220,highlighted
4,4915,62935,177676,1364951,highlighted


In [9]:
df_att.head()

,task_response_id,match_id,evaluator_id,player_id,position_id,analysis_id,attribute_id,attribute_value
0,4915,62935,177676,1361061,2,12818495,4322,56.0
1,4915,62935,177676,1361061,2,12818495,4323,56.0
2,4915,62935,177676,1361061,2,12818495,4324,67.0
3,4915,62935,177676,1361061,2,12818495,4325,56.0
4,4915,62935,177676,1361061,2,12818495,4326,45.0


In [8]:
df_att.isnull().sum()

task_response_id    0
match_id            0
evaluator_id        0
player_id           0
position_id         0
analysis_id         0
attribute_id        0
attribute_value     0
dtype: int64

In [10]:
df=pd.merge(df_att,df_pot, how="right", on=["task_response_id", "match_id","evaluator_id","player_id"])
df.head()

,task_response_id,match_id,evaluator_id,player_id,position_id,analysis_id,attribute_id,attribute_value,potential_label
0,4915,62935,177676,1361061,2,12818495,4322,56.0,average
1,4915,62935,177676,1361061,2,12818495,4323,56.0,average
2,4915,62935,177676,1361061,2,12818495,4324,67.0,average
3,4915,62935,177676,1361061,2,12818495,4325,56.0,average
4,4915,62935,177676,1361061,2,12818495,4326,45.0,average


In [11]:
df.drop(df[df["position_id"]==1].index,inplace=True)

In [12]:
df.drop(df[df.potential_label=="below_average"].index, inplace=True)

In [13]:
table=pd.pivot_table(df, values="attribute_value", index=["player_id","position_id","potential_label"], columns="attribute_id")
table

attribute_id                           4322  4323  4324  4325  4326  4327  \
player_id position_id potential_label                                       
1355710   7           average          50.5  50.5  34.0  50.5  45.0  45.0   
1356362   9           average          67.0  67.0  67.0  67.0  67.0  67.0   
1356375   3           average          67.0  67.0  67.0  67.0  67.0  67.0   
          4           average          67.0  78.0  67.0  67.0  67.0  78.0   
1356411   9           average          67.0  67.0  78.0  78.0  67.0  67.0   
...                                     ...   ...   ...   ...   ...   ...   
1907347   5           average          67.0  78.0  78.0  56.0  78.0  67.0   
          6           average          67.0  67.0  67.0  67.0  67.0  67.0   
1909532   10          average          56.0  45.0  45.0  45.0  56.0  56.0   
1909724   2           highlighted      78.0  78.0  78.0  78.0  78.0  78.0   
1909728   7           highlighted      78.0  67.0  67.0  78.0  67.0  67.0   

attribute_id                           4328  4329  4330  4332  ...  4352  \
player_id position_id potential_label                          ...         
1355710   7           average          45.0  45.0  50.5  56.0  ...  56.0   
1356362   9           average          67.0  67.0  67.0  67.0  ...  67.0   
1356375   3           average          67.0  78.0  67.0  67.0  ...  67.0   
          4           average          78.0  78.0  56.0  67.0  ...  78.0   
1356411   9           average          67.0  67.0  89.0  78.0  ...  67.0   
...                                     ...   ...   ...   ...  ...   ...   
1907347   5           average          78.0  67.0  56.0  78.0  ...  67.0   
          6           average          67.0  78.0  67.0  67.0  ...  67.0   
1909532   10          average          56.0  56.0  23.0  23.0  ...  45.0   
1909724   2           highlighted      78.0  78.0  67.0  67.0  ...  78.0   
1909728   7           highlighted      67.0  78.0  67.0  67.0  ...  67.0   

attribute_id                           4353  4354  4355  4356  4357  4407  \
player_id position_id potential_label                                       
1355710   7           average          34.0  39.5  50.5  34.0  34.0  56.0   
1356362   9           average          67.0  78.0  67.0  67.0  67.0  67.0   
1356375   3           average          67.0  78.0  67.0  67.0  67.0  67.0   
          4           average          67.0  67.0  67.0  56.0  67.0  56.0   
1356411   9           average          56.0  67.0  67.0  56.0  67.0  89.0   
...                                     ...   ...   ...   ...   ...   ...   
1907347   5           average          67.0  67.0  67.0  56.0  67.0  78.0   
          6           average          67.0  78.0  67.0  67.0  67.0  67.0   
1909532   10          average          45.0  56.0  34.0  34.0  56.0  34.0   
1909724   2           highlighted      78.0  78.0  67.0  67.0  67.0  78.0   
1909728   7           highlighted      67.0  78.0  67.0  67.0  67.0  78.0   

attribute_id                           4408  4423  4426  
player_id position_id potential_label                    
1355710   7           average          34.0  34.0  56.0  
1356362   9           average          67.0  56.0  78.0  
1356375   3           average          67.0  56.0  78.0  
          4           average          67.0  45.0  56.0  
1356411   9           average          56.0  67.0  78.0  
...                                     ...   ...   ...  
1907347   5           average          78.0  78.0  56.0  
          6           average          67.0  67.0  78.0  
1909532   10          average          34.0  34.0  23.0  
1909724   2           highlighted      78.0  67.0  78.0  
1909728   7           highlighted      67.0  67.0  78.0  

[271 rows x 34 columns]

In [14]:
table.reset_index(inplace=True)

In [15]:
table=table.astype(str)

In [16]:
table

attribute_id,player_id,position_id,potential_label,4322,4323,4324,4325,4326,4327,4328,...,4352,4353,4354,4355,4356,4357,4407,4408,4423,4426
0,1355710,7,average,50.5,50.5,34.0,50.5,45.0,45.0,45.0,...,56.0,34.0,39.5,50.5,34.0,34.0,56.0,34.0,34.0,56.0
1,1356362,9,average,67.0,67.0,67.0,67.0,67.0,67.0,67.0,...,67.0,67.0,78.0,67.0,67.0,67.0,67.0,67.0,56.0,78.0
2,1356375,3,average,67.0,67.0,67.0,67.0,67.0,67.0,67.0,...,67.0,67.0,78.0,67.0,67.0,67.0,67.0,67.0,56.0,78.0
3,1356375,4,average,67.0,78.0,67.0,67.0,67.0,78.0,78.0,...,78.0,67.0,67.0,67.0,56.0,67.0,56.0,67.0,45.0,56.0
4,1356411,9,average,67.0,67.0,78.0,78.0,67.0,67.0,67.0,...,67.0,56.0,67.0,67.0,56.0,67.0,89.0,56.0,67.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,1907347,5,average,67.0,78.0,78.0,56.0,78.0,67.0,78.0,...,67.0,67.0,67.0,67.0,56.0,67.0,78.0,78.0,78.0,56.0
267,1907347,6,average,67.0,67.0,67.0,67.0,67.0,67.0,67.0,...,67.0,67.0,78.0,67.0,67.0,67.0,67.0,67.0,67.0,78.0
268,1909532,10,average,56.0,45.0,45.0,45.0,56.0,56.0,56.0,...,45.0,45.0,56.0,34.0,34.0,56.0,34.0,34.0,34.0,23.0
269,1909724,2,highlighted,78.0,78.0,78.0,78.0,78.0,78.0,78.0,...,78.0,78.0,78.0,67.0,67.0,67.0,78.0,78.0,67.0,78.0


In [17]:
le=LabelEncoder()
table["potential_label"]=le.fit_transform(table["potential_label"])

In [18]:
table.head()

attribute_id,player_id,position_id,potential_label,4322,4323,4324,4325,4326,4327,4328,...,4352,4353,4354,4355,4356,4357,4407,4408,4423,4426
0,1355710,7,0,50.5,50.5,34.0,50.5,45.0,45.0,45.0,...,56.0,34.0,39.5,50.5,34.0,34.0,56.0,34.0,34.0,56.0
1,1356362,9,0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,...,67.0,67.0,78.0,67.0,67.0,67.0,67.0,67.0,56.0,78.0
2,1356375,3,0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,...,67.0,67.0,78.0,67.0,67.0,67.0,67.0,67.0,56.0,78.0
3,1356375,4,0,67.0,78.0,67.0,67.0,67.0,78.0,78.0,...,78.0,67.0,67.0,67.0,56.0,67.0,56.0,67.0,45.0,56.0
4,1356411,9,0,67.0,67.0,78.0,78.0,67.0,67.0,67.0,...,67.0,56.0,67.0,67.0,56.0,67.0,89.0,56.0,67.0,78.0


In [19]:
num_cols=table.columns[3:]
num_cols

Index([4322, 4323, 4324, 4325, 4326, 4327, 4328, 4329, 4330, 4332, 4333, 4335,
       4338, 4339, 4340, 4341, 4342, 4343, 4344, 4345, 4348, 4349, 4350, 4351,
       4352, 4353, 4354, 4355, 4356, 4357, 4407, 4408, 4423, 4426],
      dtype='object', name='attribute_id')

In [20]:
ss=StandardScaler()
table[num_cols]=ss.fit_transform(table[num_cols])

In [21]:
table

attribute_id,player_id,position_id,potential_label,4322,4323,4324,4325,4326,4327,4328,...,4352,4353,4354,4355,4356,4357,4407,4408,4423,4426
0,1355710,7,0,-0.542606,-0.559398,-1.404764,-0.437916,-0.766602,-0.794802,-0.907351,...,0.012848,-1.281899,-1.132245,-0.494569,-1.234869,-1.519578,-0.142650,-1.487256,-0.955381,-0.252741
1,1356362,9,0,0.595327,0.560776,0.678677,0.682837,0.722639,0.722929,0.601190,...,0.786581,0.813645,1.052769,0.631718,0.914913,0.768231,0.529664,0.668713,0.403717,1.041611
2,1356375,3,0,0.595327,0.560776,0.678677,0.682837,0.722639,0.722929,0.601190,...,0.786581,0.813645,1.052769,0.631718,0.914913,0.768231,0.529664,0.668713,0.403717,1.041611
3,1356375,4,0,0.595327,1.307560,0.678677,0.682837,0.722639,1.481795,1.355461,...,1.560314,0.813645,0.428479,0.631718,0.198319,0.768231,-0.142650,0.668713,-0.275832,-0.252741
4,1356411,9,0,0.595327,0.560776,1.373158,1.430006,0.722639,0.722929,0.601190,...,0.786581,0.115130,0.428479,0.631718,0.198319,0.768231,1.874292,-0.049944,1.083267,1.041611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,1907347,5,0,0.595327,1.307560,1.373158,-0.064332,1.467260,0.722929,1.355461,...,0.786581,0.813645,0.428479,0.631718,0.198319,0.768231,1.201978,1.387369,1.762816,-0.252741
267,1907347,6,0,0.595327,0.560776,0.678677,0.682837,0.722639,0.722929,0.601190,...,0.786581,0.813645,1.052769,0.631718,0.914913,0.768231,0.529664,0.668713,1.083267,1.041611
268,1909532,10,0,-0.163295,-0.932790,-0.710283,-0.811501,-0.021981,-0.035936,-0.153081,...,-0.760885,-0.583384,-0.195810,-1.620856,-1.234869,0.005628,-1.487277,-1.487256,-0.955381,-2.194269
269,1909724,2,1,1.353950,1.307560,1.373158,1.430006,1.467260,1.481795,1.355461,...,1.560314,1.512160,1.052769,0.631718,0.914913,0.768231,1.201978,1.387369,1.083267,1.041611


potential_label is our target variable. We want to predict potential_label so y will be potential_label.

In [26]:
y=table["potential_label"]
x=table[num_cols]

KNN with holdout method

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=5, test_size=0.3, shuffle=True)

In [34]:
knn_model_h=KNeighborsClassifier().fit(x_train,y_train)

In [36]:
y_pred=knn_model_h.predict(x_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0])

In [39]:
print("accuracy: ", accuracy_score(y_pred,y_test))
print("roc_auc: ", roc_auc_score(y_pred,y_test))
print("f1 score: ", f1_score(y_pred,y_test))
print("precision: ", precision_score(y_pred,y_test))
print("recall: ", recall_score(y_pred,y_test))

accuracy:  0.8902439024390244
roc_auc:  0.9415584415584415
f1 score:  0.5263157894736842
precision:  0.35714285714285715
recall:  1.0


KNN with cross validation and hyperparameter optimization

In [40]:
knn_model_ch=KNeighborsClassifier()
knn_model_ch.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [41]:
params={"n_neighbors": range(2,50)}

In [43]:
knn_grid=GridSearchCV(knn_model_ch,params,cv=10,n_jobs=-1,verbose=1).fit(x,y)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


In [44]:
knn_grid.best_params_

{'n_neighbors': 7}

In [53]:
knn_final=knn_model_ch.set_params(**knn_grid.best_params_).fit(x,y)
knn_final

KNeighborsClassifier(n_neighbors=7)

In [54]:
cv_results=cross_validate(knn_final,x,y,cv=5,scoring=["accuracy","f1","roc_auc"])
cv_results

{'fit_time': array([0.00429559, 0.00326037, 0.00360298, 0.00362301, 0.02134967]),
 'score_time': array([0.03620315, 0.04856753, 0.01223373, 0.02518296, 0.04223108]),
 'test_accuracy': array([0.87272727, 0.85185185, 0.85185185, 0.85185185, 0.87037037]),
 'test_f1': array([0.58823529, 0.42857143, 0.5       , 0.42857143, 0.53333333]),
 'test_roc_auc': array([0.67635659, 0.87737844, 0.76849894, 0.85412262, 0.84566596])}

In [52]:
print(cv_results["test_accuracy"].mean())
print(cv_results["test_f1"].mean())
print(cv_results["test_roc_auc"].mean())

0.8597306397306397
0.49574229691876753
0.8044045102184636
